In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import logging
logging.basicConfig(filename="./sales_table_log.log", level=logging.INFO)

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_sturday_ty = datetime.date(2020,8,1)
first_week_ty=datetime.date(2020,5,3)

last_sturday_ly = last_sturday_ty-datetime.timedelta(days=52*7)
first_week_ly=first_week_ty-datetime.timedelta(days=52*7)

df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20200801-134534-636.txt",
                            sep="|",dtype=str,usecols=['division_id','class_code_id','subclass_id'])

In [2]:
last_sturday_ty,first_week_ty,last_sturday_ly,first_week_ly

(datetime.date(2020, 8, 1),
 datetime.date(2020, 5, 3),
 datetime.date(2019, 8, 3),
 datetime.date(2019, 5, 5))

In [3]:
list_POS_this_year=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
list_POS_this_year=[x for x in list_POS_this_year if "daily" in x.lower()]

list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ty)]
list_POS_this_year=[x for x in list_POS_this_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ty)]

# 
list_POS_last_year=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_POS_last_year=[x for x in list_POS_last_year if "daily" in x.lower()]

list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]>=str(first_week_ly)]
list_POS_last_year=[x for x in list_POS_last_year if x.split("/MediaStorm_")[1][:10]<=str(last_sturday_ly)]

In [4]:
df_2020=pd.DataFrame()

for file in list_POS_this_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df=df.groupby(['division_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_end_dt']=week_end_dt
    df_2020=df_2020.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2020-05-09 2020-08-18 20:02:55.631454
2020-05-16 2020-08-18 20:03:51.943403
2020-05-23 2020-08-18 20:05:30.732448
2020-05-30 2020-08-18 20:07:01.923102
2020-06-06 2020-08-18 20:08:16.051127
2020-06-20 2020-08-18 20:09:30.489220
2020-06-13 2020-08-18 20:11:02.411526
2020-06-27 2020-08-18 20:12:14.061497
2020-07-04 2020-08-18 20:14:03.229875
2020-07-11 2020-08-18 20:15:24.834209
2020-07-18 2020-08-18 20:16:59.161615
2020-07-25 2020-08-18 20:18:11.783229
2020-08-01 2020-08-18 20:19:54.967237


In [5]:
df_2019=pd.DataFrame()
df_prod_mapping=pd.read_csv("/home/jian/BigLots/static_files/ProductTaxonomy/MediaStormProductTaxonomy20190401-134939-109.txt",
                            sep="|",dtype=str,usecols=['division_id','class_code_id','subclass_id'])
for file in list_POS_last_year:
    df=pd.read_csv(file,dtype=str,sep="|")
    df=pd.merge(df,df_prod_mapping,on=['class_code_id','subclass_id'],how="left")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['division_id']=df['division_id'].fillna("-1")
    df['rewards_label']=np.where(pd.isnull(df['customer_id_hashed']),"non_rewards","rewards")
    df['store_label']=np.where(df['location_id']=="6990","online","instore")
    week_end_dt=df['transaction_dt'].max()
    
    df=df.groupby(['division_id','rewards_label','store_label'])['item_transaction_amt'].sum().to_frame().reset_index()
    df['week_end_dt']=week_end_dt
    df_2019=df_2019.append(df)
    print(week_end_dt,datetime.datetime.now())
    

2019-05-11 2020-08-18 20:20:56.522358
2019-05-18 2020-08-18 20:22:02.763073
2019-05-25 2020-08-18 20:23:42.644400
2019-06-01 2020-08-18 20:25:02.133425
2019-06-08 2020-08-18 20:26:16.847426
2019-06-15 2020-08-18 20:27:38.163022
2019-06-22 2020-08-18 20:28:43.275657
2019-06-29 2020-08-18 20:30:08.982239
2019-07-06 2020-08-18 20:31:35.970238
2019-07-13 2020-08-18 20:33:11.319980
2019-07-20 2020-08-18 20:34:14.705651
2019-07-27 2020-08-18 20:35:02.724867
2019-08-03 2020-08-18 20:36:02.355055


In [6]:
print("df_2019.shape",df_2019.shape)
print("df_2020.shape",df_2020.shape)

df_2019['division_id']=df_2019['division_id'].astype(int)
df_2020['division_id']=df_2020['division_id'].astype(int)

df_2019=df_2019.sort_values(['division_id','rewards_label','store_label'])
df_2020=df_2020.sort_values(['division_id','rewards_label','store_label'])


df_2019.shape (417, 5)
df_2020.shape (407, 5)


In [7]:
df_2019_wide=df_2019.pivot_table(index=['division_id','rewards_label','store_label'],columns="week_end_dt",values="item_transaction_amt").reset_index()
df_2020_wide=df_2020.pivot_table(index=['division_id','rewards_label','store_label'],columns="week_end_dt",values="item_transaction_amt").reset_index()
df_2019_wide.insert(0,'year',"2019")
df_2020_wide.insert(0,'year',"2020")

In [8]:
division_name=pd.read_table("/home/jian/BigLots/static_files/MediaStorm Data Extract - Division Names.txt",sep="|")
division_name

,division_id,division_desc
0,1,FOOD
1,2,CONSUMABLES
2,3,SEASONAL
3,4,SOFTHOME
4,5,HARDHOME
5,6,ELECNACCES
6,7,Division 7
7,8,DIVISION 8 (EMP DISCOUNT)
8,9,FURNITURE
9,10,DIVISION 10


In [9]:
writer=pd.ExcelWriter("./BL_sales_by_division_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_2019_wide.to_excel(writer,"2019_wide_all",index=False)
df_2020_wide.to_excel(writer,"2020_wide_all",index=False)
division_name.to_excel(writer,"div_name",index=False)
df_2019.to_excel(writer,"2019_long_all",index=False)
df_2020.to_excel(writer,"2020_long_all",index=False)
for comb,group in df_2019_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
    
for comb,group in df_2020_wide.groupby(['rewards_label','store_label']):
    year=group['year'].unique().tolist()[0]
    tab_name="_".join(comb)+"_"+year
    group.to_excel(writer,tab_name,index=False)
writer.save()